[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainNNmodel.ipynb)

In [23]:
!pip install tqdm

In [0]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from tqdm import tqdm

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/
%ls

/content/gdrive/My Drive/projects
model_LGB.sav  model_XGB.sav  user_train_wip.csv


In [0]:
s_data = 'user_train_wip.csv'
s_col_tgt = 'totals.transactionRevenue_sum'
l_col_drop = ['date_min', 'date_max',s_col_tgt]
filename = 'model_NN.sav'

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 12.5 s, sys: 2.83 s, total: 15.3 s
Wall time: 17.4 s


In [14]:
%%time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_x_n = pd.DataFrame(sc.fit_transform(df_x.fillna(0)),columns=df_x.columns)
df_x_n

CPU times: user 2.98 s, sys: 38 ms, total: 3.02 s
Wall time: 3.02 s


In [0]:
def build_model(N):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(N,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [0]:
%%time
k = 2                     #4
epochs = 1                #100

val_size = df_x.shape[0] // k
all_mae = []

for i in tqdm(range(k)):
    print('fold',i+1)
    
    val_x = df_x[i*val_size:(i+1)*val_size]
    val_y = df_y[i*val_size:(i+1)*val_size]
    
    train_x = pd.concat([df_x[:i*val_size], df_x[(i+1)*val_size:]], axis=0)
    train_y =  pd.concat([df_y[:i*val_size], df_y[(i+1)*val_size:]], axis=0)
    
    model = build_model(df_x.shape[1])
    
    history = model.fit(train_x,
                        train_y,
                        validation_data=(val_x, val_y),
                        epochs=epochs,
                        batch_size=1,
                        verbose=0)
    all_mae.append(history.history['val_mean_absolute_error'])

  0%|          | 0/2 [00:00<?, ?it/s]

fold 1


In [0]:
average_mae = np.mean(all_mae, axis=0)

In [0]:
epoch = range(1, epochs+1)

plt.plot(epoch, average_mae, label='Validation mae')
plt.xlabel('epoch')
plt.ylabel('mae')
plt.grid(True)
plt.legend()
plt.show()